In [26]:
from numpy import loadtxt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report, precision_score, accuracy_score
from sklearn.model_selection import StratifiedKFold as kfold
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
import pandas as pd
import matplotlib.pyplot as plt
import csv
import numpy as np
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
data = pd.read_csv('/content/drive/MyDrive/nn_data.txt')
data.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [28]:
unscaled_x = data.iloc[:,0:8]
unscaled_y = data.iloc[:,8]

In [29]:
model = Sequential()
model.add(Dense(8, input_shape=(8,), activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(unscaled_x, unscaled_y, epochs=100, batch_size=10)
_, accuracy = model.evaluate(unscaled_x, unscaled_y)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
77/77 [==============================] - 1s 1ms/step - loss: 7.6844 - accuracy: 0.5528
Epoch 2/100
77/77 [==============================] - 0s 1ms/step - loss: 2.0299 - accuracy: 0.6441
Epoch 3/100
77/77 [==============================] - 0s 1ms/step - loss: 1.0460 - accuracy: 0.6206
Epoch 4/100
77/77 [==============================] - 0s 1ms/step - loss: 0.7835 - accuracy: 0.6441
Epoch 5/100
77/77 [==============================] - 0s 1ms/step - loss: 0.7118 - accuracy: 0.6584
Epoch 6/100
77/77 [==============================] - 0s 1ms/step - loss: 0.6764 - accuracy: 0.6714
Epoch 7/100
77/77 [==============================] - 0s 1ms/step - loss: 0.6523 - accuracy: 0.6741
Epoch 8/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6253 - accuracy: 0.6845
Epoch 9/100
77/77 [==============================] - 0s 1ms/step - loss: 0.6154 - accuracy: 0.6832
Epoch 10/100
77/77 [==============================] - 0s 1ms/step - loss: 0.6099 - accuracy: 0.6767
Epoch 11/

In [30]:
min_max_scaling = preprocessing.MinMaxScaler().fit(data)
scaled_data = min_max_scaling.transform(data)

In [32]:
scaled_x = scaled_data[:,0:8]
scaled_y = scaled_data[:,8]

In [33]:
kfold_splits = 10
folds = kfold(n_splits=kfold_splits)

In [34]:
precision_scores = []
accuracy_scores = []

## With k-fold validation

In [35]:
def ann_model_with_k_fold(train_input,train_output,test_input,test_output,epochs,batch_size):
  ann_model = Sequential()
  ann_model.add(Dense(8, input_shape=(8,), activation='relu'))
  ann_model.add(Dense(6, activation='relu'))
  ann_model.add(Dense(1, activation='sigmoid'))
  ann_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  ann_model.fit(train_input, train_output, epochs=epochs, batch_size=batch_size)
  y_pred = (ann_model.predict(test_input) > 0.5).astype(int)
  precision_scores.append(precision_score(test_output, y_pred))
  accuracy_scores.append(accuracy_score(test_output, y_pred))

## Without k-fold validation

In [36]:
def ann_model_without_k_fold(X,y,epochs,batch_size) :
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
    ann_model = Sequential();
    ann_model.add(Dense(8, input_shape=(8,), activation='relu'))
    ann_model.add(Dense(6, activation='relu'))
    ann_model.add(Dense(1, activation='sigmoid'))
    ann_model.compile( optimizer = 'adam' , loss = 'binary_crossentropy', metrics = ['accuracy'])
    ann_model.fit(X_train,y_train,epochs=epochs,batch_size = batch_size)
    y_pre = ann_model.predict(X_test);
    y_pred = [] ;
    for element in y_pre:
        if element > 0.5 :
            y_pred.append(1)
        else :
            y_pred.append(0)
    precisionScore = precision_score(y_test, y_pred)
    accuracyScore = accuracy_score(y_test, y_pred)
    return precisionScore,accuracyScore ;

## Summary

In [37]:
u_precisionScore,u_accuracyScore = ann_model_without_k_fold(unscaled_x,unscaled_y,100,10)
s_precisionScore,s_accuracyScore = ann_model_without_k_fold(scaled_x,scaled_y,100,10)

Epoch 1/100
54/54 [==============================] - 1s 1ms/step - loss: 7.4824 - accuracy: 0.3769
Epoch 2/100
54/54 [==============================] - 0s 1ms/step - loss: 3.3417 - accuracy: 0.4086
Epoch 3/100
54/54 [==============================] - 0s 1ms/step - loss: 1.6994 - accuracy: 0.5541
Epoch 4/100
54/54 [==============================] - 0s 1ms/step - loss: 0.9750 - accuracy: 0.6325
Epoch 5/100
54/54 [==============================] - 0s 1ms/step - loss: 0.7238 - accuracy: 0.6437
Epoch 6/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6774 - accuracy: 0.6455
Epoch 7/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6631 - accuracy: 0.6455
Epoch 8/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6516 - accuracy: 0.6567
Epoch 9/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6441 - accuracy: 0.6549
Epoch 10/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6414 - accuracy: 0.6660
Epoch 11/

In [38]:
print("Without k-fold,")
print("Precision (Unscaled) :  %.2f%% " % (u_precisionScore))
print("Accuracy (Unscaled) :  %.2f%% " % (u_accuracyScore))
print("Precision (Scaled) :  %.2f%% " % (s_precisionScore))
print("Accuracy (Scaled) :  %.2f%% " % (s_accuracyScore))

Without k-fold,
Precision (Unscaled) :  0.52% 
Accuracy (Unscaled) :  0.66% 
Precision (Scaled) :  0.68% 
Accuracy (Scaled) :  0.75% 


In [39]:
for train_index, test_index in folds.split(scaled_x,scaled_y):
   X_train, X_test = scaled_x[train_index], scaled_x[test_index]
   y_train, y_test = scaled_y[train_index], scaled_y[test_index]
   ann_model_with_k_fold(X_train,y_train,X_test,y_test,100,10)

Epoch 1/100
69/69 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5870
Epoch 2/100
69/69 [==============================] - 0s 2ms/step - loss: 0.6773 - accuracy: 0.6522
Epoch 3/100
69/69 [==============================] - 0s 2ms/step - loss: 0.6658 - accuracy: 0.6522
Epoch 4/100
69/69 [==============================] - 0s 1ms/step - loss: 0.6590 - accuracy: 0.6522
Epoch 5/100
69/69 [==============================] - 0s 1ms/step - loss: 0.6540 - accuracy: 0.6522
Epoch 6/100
69/69 [==============================] - 0s 1ms/step - loss: 0.6494 - accuracy: 0.6522
Epoch 7/100
69/69 [==============================] - 0s 1ms/step - loss: 0.6444 - accuracy: 0.6522
Epoch 8/100
69/69 [==============================] - 0s 1ms/step - loss: 0.6391 - accuracy: 0.6522
Epoch 9/100
69/69 [==============================] - 0s 1ms/step - loss: 0.6335 - accuracy: 0.6522
Epoch 10/100
69/69 [==============================] - 0s 1ms/step - loss: 0.6274 - accuracy: 0.6536
Epoch 11/

In [40]:
print("With k-fold,")
print("Average Precision (K-Fold) :  %.2f%% (+/- %.2f%%)" % (np.mean(precision_scores), np.std(precision_scores)))
print("Average Accuracy (K-Fold) : %.2f%% (+/- %.2f%%)" % (np.mean(accuracy_scores), np.std(accuracy_scores)))

With k-fold,
Average Precision (K-Fold) :  0.72% (+/- 0.08%)
Average Accuracy (K-Fold) : 0.77% (+/- 0.04%)
